# Pytorchの導入用ノートブック

(2023/10/27　M.Udagawa)

基本はGoogle colabを想定。ライブラリが揃っていればVScodeなどでも動く。が、ほとんど空欄。

セミナーなどで発表前に配布し、穴埋めしながら進めることを前提としている。

# ライブラリのインポート

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# Pytorchにおけるベクトル、行列、多次元配列

## (基本)torch.tensor

### ベクトル



$\mathbf{v} = [0, 1] $

を作ってみる

In [3]:
torch_v = torch.tensor([0,1])
print(torch_v)

tensor([0, 1])


### 行列



$M = \begin{pmatrix}0& 1 \\ 2& 3\end{pmatrix}$

In [9]:
torch_M =torch.tensor([[0,1],[2,3]])
print(torch_M)

tensor([[0, 1],
        [2, 3]])


### 多次元配列

完全反対称テンソル（エディントンのイプシロン）

$\epsilon_{ijk} = \begin{cases} 1 & (i,j,k)が(1,2,3)の偶置換 \\ -1&(i,j,k)が(1,2,3)の奇置換\\ 0 & other \end{cases}$

In [10]:
torch_e = torch.tensor([[[0,0,0],[0,0,1],[0,-1,0]],[[0,0,-1],[0,0,0],[1,0,0]],[[0,1,0],[-1,0,0],[0,0,0]]])
print(torch_e)

tensor([[[ 0,  0,  0],
         [ 0,  0,  1],
         [ 0, -1,  0]],

        [[ 0,  0, -1],
         [ 0,  0,  0],
         [ 1,  0,  0]],

        [[ 0,  1,  0],
         [-1,  0,  0],
         [ 0,  0,  0]]])


In [11]:
torch_e = torch.zeros(3,3,3)
for i in range(3):
  for j in range(3):
    for k in range(3):
      if (i!=j) and (j!=k) and (k!=i):
        #p:permutation(置換)
        #inv:inversion(転倒数)
        #sgn:sign(signature):置換の符号（符号数）
        p = [i,j,k]
        inv = 0
        if p[0]>p[1]:
          inv+=1
        if p[0]>p[2]:
          inv+=1
        if p[1]>p[2]:
          inv+=1
        sgn = (-1) ** inv
        torch_e[i,j,k]=int(sgn)
print(torch_e)

tensor([[[ 0.,  0.,  0.],
         [ 0.,  0.,  1.],
         [ 0., -1.,  0.]],

        [[ 0.,  0., -1.],
         [ 0.,  0.,  0.],
         [ 1.,  0.,  0.]],

        [[ 0.,  1.,  0.],
         [-1.,  0.,  0.],
         [ 0.,  0.,  0.]]])


In [12]:
torch_e = torch.zeros(3,3,3)

for i in range(3):
  for j in range(3):
    for k in range(3):
      torch_e[i,j,k] = np.sign(j-i)*np.sign(k-i)*np.sign(k-j)
print(torch_e)

tensor([[[ 0.,  0.,  0.],
         [ 0.,  0.,  1.],
         [ 0., -1.,  0.]],

        [[ 0.,  0., -1.],
         [ 0.,  0.,  0.],
         [ 1.,  0.,  0.]],

        [[ 0.,  1.,  0.],
         [-1.,  0.,  0.],
         [ 0.,  0.,  0.]]])


## (基本)numpy配列からtorch.tensorへの移行

実際のプログラムではnumpy配列をメインに扱い、ANNへ流す時だけtorch.tensorにすることがほとんど。この操作を扱ってみる。

In [14]:
np_e = np.zeros(3**3).reshape(3,3,3)

for i in range(3):
  for j in range(3):
    for k in range(3):
      np_e[i,j,k] = np.sign(j-i)*np.sign(k-i)*np.sign(k-j)

print(np_e, np_e.shape)

[[[ 0.  0.  0.]
  [ 0.  0.  1.]
  [ 0. -1.  0.]]

 [[ 0.  0. -1.]
  [ 0.  0.  0.]
  [ 1.  0.  0.]]

 [[ 0.  1.  0.]
  [-1.  0.  0.]
  [ 0.  0.  0.]]] (3, 3, 3)


numpyをtorch.tensorにするには？

In [15]:
torch_e =torch.tensor(np_e)
print(torch_e, torch_e.shape)

tensor([[[ 0.,  0.,  0.],
         [ 0.,  0.,  1.],
         [ 0., -1.,  0.]],

        [[ 0.,  0., -1.],
         [ 0.,  0.,  0.],
         [ 1.,  0.,  0.]],

        [[ 0.,  1.,  0.],
         [-1.,  0.,  0.],
         [ 0.,  0.,  0.]]], dtype=torch.float64) torch.Size([3, 3, 3])


# ANNの実装(出力まで)

今回は隠れ層(hidden layer)が一層の全結合ネットワーク(fully-connected neural netwark)を作成する

pytorchでのネットワークの実装は'Define by run'。詳しくは省略するが、

classで概形決める->インスタンス生成時の引数で構造が決まる->利用

と考えておけばOK。(TensorFlow,Kerasでは'Define by run', 'Define and run'のどちらも可)

まずはクラスの作成

最後の行でインスタンスを生成

In [26]:
#input_size:入力（簡単のためベクトルを推定）の大きさ
#hidden_size:隠れ層の大きさ
#output_size:出力（ベクトル）の大きさ

class netwark(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()

        self.fcnn1 = nn.Linear(input_size, hidden_size, bias = False)
        self.fcnn2 = nn.Linear(hidden_size, output_size, bias = False)

    def forward(self, x):
        output = self.fcnn1(x)
        output = self.fcnn2(output)
        output = F.tanh(output)

        return output

In [38]:
input_size = 5
hidden_size = 20
output_size = 2
model = netwark(input_size=input_size,hidden_size=hidden_size, output_size=output_size)

実際に入力を作って出力させてみよう!

In [39]:
num = torch.arange(input_size, dtype = torch.float32)

print(model(num))

tensor([0.1313, 0.2618], grad_fn=<TanhBackward0>)


ndarrayでも動く？

In [33]:
num = np.arange(input_size, dtype = np.float32)
num = torch.tensor(num)

print(model(num))

tensor([-0.7047, -0.0742], grad_fn=<TanhBackward0>)
